In [ ]:
import sys
import os
from pathlib import Path

# Identificar raiz do projeto automaticamente
def get_project_root():
    current = Path.cwd()
    while current.name != 'analise-temporal-municipios' and current.parent != current:
        current = current.parent
    return current

ROOT = get_project_root()
print(f"Project Root: {ROOT}")

# Adicionar raiz ao sys.path para permitir imports de 'src'
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Mudar diretório de trabalho para a raiz para facilitar caminhos relativos (data/...)
os.chdir(ROOT)
print(f"Working Directory defined to: {os.getcwd()}")


## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [ ]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
from pysus import SIH
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geobr
import os
from pathlib import Path
from typing import List, Tuple

import sys
from src.merradownload.merra_scraping import baixar_merra

## 01 - Baixar shapefiles

In [ ]:
import os
from pathlib import Path
from ipywidgets import Dropdown, Button, Output, VBox
from IPython.display import display
import pandas as pd
import geobr
import matplotlib.pyplot as plt

# Paths corrigidos (relativos à raiz)
municipios_coord_df = pd.read_csv("data/utils/municipios_coord.csv", sep=";")
estados = sorted(municipios_coord_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(municipios_coord_df[municipios_coord_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(options=estados, description='Estado:')
municipios_dd = Dropdown(options=municipios_por_estado[estados[0]], description='Município:')
baixar_btn = Button(description="Baixar e Plotar Shapefile")
output_mapa = Output()

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

def baixar_plotar_shapefile(b):
    uf = estados_dd.value
    municipio = municipios_dd.value

    with output_mapa:
        output_mapa.clear_output()
        print(f"Baixando shapefile para {municipio}, {uf}...")
        try:
            # Lookup Code
            localizar_municipio = geobr.lookup_muni(name_muni=municipio)
            localizar_municipio = localizar_municipio[localizar_municipio['abbrev_state'] == uf]
            if localizar_municipio.empty:
                 print(f"Município {municipio}-{uf} não encontrado no geobr.")
                 return
            
            code_muni = localizar_municipio.iloc[0]['code_muni']
            gdf = geobr.read_municipality(code_muni=code_muni)
            print("Download concluído.")

            # Save Logic
            save_dir = f"data/shapefiles/{uf}-{municipio.replace(' ', '_')}/"
            os.makedirs(save_dir, exist_ok=True)
            
            filepath = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}.shp")
            gdf.to_file(filepath, driver='ESRI Shapefile')
            print(f"Shapefile salvo em: {filepath}")

            # Save IBGE CSV
            try:
                ibge_df = pd.read_csv("data/utils/municipios_ibge.csv")
                # Handle code types
                city_ibge_info = ibge_df[ibge_df['codigo_ibge'] == float(code_muni)]
                
                if city_ibge_info.empty:
                    city_ibge_info = ibge_df[ibge_df['Nome_Município'] == municipio]
                
                if not city_ibge_info.empty:
                    csv_path = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}_ibge.csv")
                    city_ibge_info.to_csv(csv_path, index=False)
                    print(f"CSV IBGE salvo em: {csv_path}")
                else:
                    print("Aviso: Não foi possível encontrar informações no arquivo municipios_ibge.csv")
            except Exception as e:
                print(f"Erro ao salvar CSV IBGE: {e}")

            # Plot
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            gdf.plot(ax=ax, facecolor='#add8e6', edgecolor='black')
            ax.set_title(f"Mapa de {municipio} - {uf}")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            plt.grid(True)
            plt.show()
            
        except Exception as e:
            print(f"Erro ao processar: {e}")

baixar_btn.on_click(baixar_plotar_shapefile)

display(VBox([estados_dd, municipios_dd, baixar_btn, output_mapa]))

## 02 - Criar arquivo final

In [ ]:
import os
import gc
from pathlib import Path
from datetime import date
from ipywidgets import Dropdown, Button, Output, VBox, BoundedIntText, HBox, Accordion, HTML, DatePicker, Layout, Label, Text
from IPython.display import display
from src.baixar_dados.download_all import download_all, build_shapefile_catalog, SUPPORTED_DISEASES

def make_download_all_widget(
    *,
    shapefiles_dir="data/shapefiles",
    default_output_dir="data/output",
    default_cache_dir="data/cache",
    default_final_csv="data/output/final/final_by_ibge_date.csv",
    default_disease="asma",
    default_final_schema="reference",
    default_schema_csv="data/utils/schema_template.csv",
    log_level="INFO",
):
    """
    UI para Jupyter Notebook (ipywidgets) atualizada.
    """
    
    # 1. Carregar catálogo
    catalog = build_shapefile_catalog(shapefiles_dir=shapefiles_dir)
    options = [("Todos os municípios (todos os shapefiles)", "__ALL__")]
    for _, row in catalog.iterrows():
        cod = str(row.get("codibge") or "").strip()
        name = str(row.get("shapefile_nome") or "").strip()
        path = str(row.get("shapefile_path") or "").strip()
        label = f"{cod} — {name}" if cod else name
        options.append((label, path))

    # 2. Widgets
    disease_dd = Dropdown(
        options=list(SUPPORTED_DISEASES),
        value=default_disease if default_disease in SUPPORTED_DISEASES else SUPPORTED_DISEASES[0],
        description="Doença:",
        style={'description_width': 'initial'}
    )
    
    # DatePicker (Calendário)
    start_picker = DatePicker(description="Início", value=date(2023, 1, 1))
    end_picker = DatePicker(description="Fim", value=date(2023, 1, 31))
    
    # Workers
    workers_input = BoundedIntText(
        value=3,
        min=1,
        max=32,
        step=1,
        description="Workers:",
        style={'description_width': 'initial'},
        layout=Layout(width='150px')
    )
    workers_warning = HTML(
        value="<span style='color:orange; font-size:0.9em;'>⚠️ Acima de 3 workers exige PC potente (muita RAM/CPU).</span>",
        layout=Layout(margin='5px 0 0 10px')
    )

    target_dd = Dropdown(
        options=options, 
        value="__ALL__", 
        description="Município:",
        style={'description_width': 'initial'}
    )
    
    # Schema
    final_schema_dd = Dropdown(
        options=["reference", "all", "inmet"],
        value=default_final_schema,
        description="Schema:",
        description_tooltip="Define quais colunas estarão no CSV final.",
        style={'description_width': 'initial'}
    )
    schema_help = HTML(
        value="<div style='font-size:0.85em; color:#555; margin-left:10px;'>"
              "<b>Schema:</b> Define a estrutura do arquivo final.<br>"
              "• <i>reference</i>: Segue o padrão do arquivo modelo (recomendado).<br>"
              "• <i>all</i>: Inclui todas as colunas baixadas.<br>"
              "• <i>inmet</i>: Apenas dados meteorológicos."
              "</div>"
    )
    
    # Caminhos (Advanced)
    output_dir_txt = Text(value=str(default_output_dir), description="Saída:")
    cache_dir_txt = Text(value=str(default_cache_dir), description="Cache:")
    final_csv_txt = Text(value=str(default_final_csv), description="Final CSV:")
    schema_csv_txt = Text(value=str(default_schema_csv), description="Schema CSV:")
    
    run_btn = Button(description="Executar Download", button_style="success", icon="download", layout=Layout(width='100%', margin='20px 0'))
    out = Output()

    def _on_run(_):
        # 1. Limpar Output e Resetar Estado Visual
        out.clear_output()
        run_btn.disabled = True
        run_btn.description = "Executando... (Aguarde)"
        
        with out:
            try:
                # 2. Garbage Collection explícito para liberar memória de execuções anteriores
                gc.collect()
                print("♻️ Memória limpa. Iniciando nova execução...")

                if not start_picker.value or not end_picker.value:
                    print("⚠️ Selecione as datas de Início e Fim.")
                    return
                
                start_str = start_picker.value.isoformat()
                end_str = end_picker.value.isoformat()
                selected = target_dd.value
                
                print(f"🚀 Configuração validada.")
                print(f"📅 Período: {start_str} a {end_str}")
                print(f"⚙️ Config: Workers={workers_input.value} | Schema={final_schema_dd.value}")
                
                if workers_input.value > 3:
                     print("⚠️ Atenção: Alto número de workers selecionado. Monitorando uso de recursos...")

                kwargs = {
                    "start": start_str,
                    "end": end_str,
                    "disease": disease_dd.value,
                    "output_dir": output_dir_txt.value,
                    "cache_dir": cache_dir_txt.value,
                    "final_csv": final_csv_txt.value,
                    "final_schema": final_schema_dd.value,
                    "schema_csv": schema_csv_txt.value or None,
                    "log_level": log_level,
                    "max_workers": workers_input.value,
                }
                
                if selected == "__ALL__":
                    df = download_all(shapefiles_dir=shapefiles_dir, **kwargs)
                else:
                    df = download_all(shapefile=selected, **kwargs)
                
                print("✅ Processo finalizado com sucesso!")
                if not df.empty:
                    print(f"📊 Dados gerados: {len(df)} linhas, {len(df.columns)} colunas.")
                    display(df.tail())
                else:
                    print("⚠️ Nenhum dado retornado (verifique logs ou disponibilidade).")
                    
            except Exception as e:
                print(f"❌ Erro na execução: {e}")
                import traceback
                traceback.print_exc()
                print("\n🔄 O processo falhou. Você pode ajustar as configurações e tentar novamente.")
            finally:
                # 3. Reabilitar botão sempre, sucesso ou falha
                run_btn.disabled = False
                run_btn.description = "Executar Download"

    run_btn.on_click(_on_run)

    # Layout
    ui = VBox([
        HTML("<h3>Configuração do Download (Orquestrador)</h3>"),
        HBox([disease_dd, target_dd]),
        HBox([start_picker, end_picker]),
        HBox([workers_input, workers_warning]),
        HBox([final_schema_dd, schema_help]),
        Accordion(children=[
            VBox([output_dir_txt, cache_dir_txt, final_csv_txt, schema_csv_txt])
        ], titles=('Opções Avançadas (Caminhos)',)),
        run_btn,
        out
    ])
    display(ui)

# Usar paths relativos à raiz
make_download_all_widget(shapefiles_dir="data/shapefiles")
